In [1]:
!git clone https://github.com/keirp/automatic_prompt_engineer.git

Cloning into 'automatic_prompt_engineer'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 240 (delta 25), reused 19 (delta 19), pack-reused 184
Receiving objects: 100% (240/240), 4.48 MiB | 8.98 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [2]:
!cd /content/automatic_prompt_engineer && pip install -e .


Obtaining file:///content/automatic_prompt_engineer
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━

# for epistemic reasoning

In [4]:
import json
import random
import re

In [5]:

file_path = "/content/automatic_prompt_engineer/data/bigbench-ii/epistemic_reasoning/task.json"

with open(file_path, 'r') as json_file:

    data_orig = json.load(json_file)

In [6]:
len(data_orig['examples'])

2000

In [7]:
data=data_orig['examples']

In [8]:
random.shuffle(data)

# Define the proportions for train, test, and validation sets
total_samples = len(data)
train_ratio = 0.7
test_ratio = 0.15
validation_ratio = 0.15

# Calculate the sizes of each set
train_size = int(total_samples * train_ratio)
test_size = int(total_samples * test_ratio)
validation_size = int(total_samples * validation_ratio)

# Split the data into sets
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
validation_data = data[train_size + test_size:]

In [9]:
train_data[:5]

[{'input': 'Premise: Joseph recognizes that Charles suspects that two women are hugging on a path through a grassy area with a cow visible past them. Hypothesis: Charles suspects that Joseph recognizes that two women are hugging on a path through a grassy area with a cow visible past them.',
  'target_scores': {'entailment': 0, 'non-entailment': 1}},
 {'input': 'Premise: Abigail knows that Taylor assumes that a gentleman standing on steps getting his picture taken by a girl wearing a dress. Hypothesis: Taylor assumes that Abigail knows that a gentleman standing on steps getting his picture taken by a girl wearing a dress.',
  'target_scores': {'entailment': 0, 'non-entailment': 1}},
 {'input': 'Premise: Charlotte assumes that David learns that a large group of protesters are walking down the street with signs. Hypothesis: Charlotte assumes that a large group of protesters are walking down the street with signs.',
  'target_scores': {'entailment': 1, 'non-entailment': 0}},
 {'input': 'P

# llama

# algorithm

In [15]:
task_description=data_orig['description']

In [16]:
task_prefix=data_orig['task_prefix']

In [23]:
def extract_answer(line):
  pattern = r'<Ans>(.*?)</Ans>'
  matches = re.findall(pattern, line)

  if matches =='entailment' or matches=='Entailment' or "non-entail" not in matches:
    return 1
  return 0


In [24]:
def run_inference_step1():
  wrong_ans_indices=[]
  for i in range(len(train_data)):
    prompt_t= task_description+'\n'+'Given input: '+train_data[i]['input']+'\n'+task_prefix+'Put your answer within tag <Ans> and </Ans>'
    # print(prompt_t)
    output = c.run(
      "meta/llama-2-70b-chat:2c1608e18606fad2812020dc541930f2d0495ce32eee50074220b87300bc16e1",
      input={"prompt": prompt_t},api_token="r8_7Q2ceKItTsIsBjoJwvIfl5NdXj7qxMZ0ch2hF"
  )

    # print(''.join(output))
    op=''.join(output)
    ans=extract_answer(op)
    ground_truth=1
    if train_data[i]['target_scores']['non-entailment']==1:
      ground_truth=0
    if (ans!=ground_truth):
      wrong_ans_indices.append(i)

  return wrong_ans_indices

In [25]:
wrong_answers=run_inference_step1()

ReplicateError: ignored

# with hugginface codebase

In [7]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root

In [15]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.2 MB/s eta 0:00:00


In [1]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Result: I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?

Comment: Yes, definitely! Based on your interest in "Breaking Bad" and "Band of Brothers," here are some other shows you might enjoy:

1. "The Sopranos" - This HBO series is a crime drama that explores the life of a New Jersey mob boss, Tony Soprano, as he navigates the criminal underworld and deals with personal and family issues.
2. "The Wire" - This HBO series is a gritty and realistic portrayal of the drug trade in Baltimore, and explores the impact it has on the city and its residents.
3. "Mad Men" - This AMC series is set in the 1960s and follows the lives of advertising executives on Madison Avenue, exploring them
